In [ ]:
import sys
sys.path.append("../src")
from utils import Utils
from landforms import Landforms
from extents import GetExtents
from processDem import ProcessDem
from rasterMasking import RasterMasking
import os
import glob

# 1. Indexing the Tiff files

In [ ]:
## Manual input list of the DEM and backscatter to be used
# input_dem =[path/to/your/dem1.tif, path/to/your/dem2.tif, ...]
# input_bs = [path/to/your/backscatter1.tif, path/to/your/backscatter2.tif, ...]

# The first listed input dem will be the basis for alignment
input_dems = [
            r"C:\Users\ageglio\Desktop\test_dem_2\TB_2\1m\TB_depth_1m.tif",
            r"C:\Users\ageglio\Desktop\test_dem_2\TB_2\halfm\TB_depth_halfm.tif",
            r"C:\Users\ageglio\Desktop\test_dem_2\TB_2\quaterm\TB_depth_quaterm.tif"
]
# Backscatter files can be empty if not available, e.g. input_bs = []
# Backscatter files should be aligned with the DEMs, i.e. same resolution and extent
# If not aligned, the script will use the first DEM as reference to align the backscatter files
# Backscatter files should have "BS" in their filename to be recognized as backscatter files
input_bss = [r"C:\Users\ageglio\Desktop\test_dem_2\TB_2\1m\TB_BS_1m.tif",
            r"C:\Users\ageglio\Desktop\test_dem_2\TB_2\halfm\TB_BS_halfm.tif",
            r"C:\Users\ageglio\Desktop\test_dem_2\TB_2\quaterm\TB_BS_quaterm.tif"
]
# flatten lists
input_rasters_list = [r for r in input_dems + input_bss if os.path.exists(r)]
# Define the products to be created
# products = ["slope", "aspect", "roughness", "tpi", "tri", "hillshade", "shannon_index"]
products = ["slope"]
print("ORIGINAL Bathy:", input_dems[0])
print("ORIGINAL backscatter:", None if not input_bss else input_bss[0])
print("PRODUCTS:", products)
GetExtents.return_min_max_tif_df([input_dems[0]])
intersection_mask, aligned_rasters = RasterMasking.return_valid_data_mask_intersection(input_rasters_list)
bss = [f for f in aligned_rasters if 'bs' in f.lower()]
dems = [f for f in aligned_rasters if 'bs' not in f.lower()]
print("Aligned DEMs:", dems)
print("Aligned Backsatter:", bss)

# 2. Generate derived DEM products

In [ ]:
# Generate terrain products of all of the dems and align the backscatter files
for input_dem, input_bs in zip(dems, bss):
    # Create an instance of the ProcessDEM class with the specified parameters
    generateHabitatDerivates = ProcessDem(
                                    input_dem=input_dem,
                                    input_bs=input_bs,
                                    binary_mask=intersection_mask,
                                    products=products,
                                    shannon_window=[3, 9, 21],
                                    fill_method="IDW",
                                    fill_iterations=1,
                                    chunk_size=None,  # Use default chunk size
                                    generate_boundary=True
                                    )
    generateHabitatDerivates.process_dem()

## 3. Create geomorphons landforms using arcpy 3.4

In [ ]:
# create original landforms from ArcGIS Pro
for input_dem in input_dems:
    filled_dem = os.path.join(os.path.dirname(input_dem), "filled", Utils.sanitize_path_to_name(input_dem) + "_filled.tif")
    Landforms(filled_dem).generate_landforms()
    # Clean up additional files after processing
    Utils.remove_additional_files(directory=os.path.dirname(input_dem))

In [ ]:
## If you want to save the landforms classification histogram and metadata
classes = "10"
input_dem = filled_dem
raster_nc = os.path.join(os.path.dirname(os.path.dirname(input_dem)), "geomorphons", Utils.sanitize_path_to_name(input_dem)+f"_{classes}c.tif")
Landforms.analyze_geomorphon_data(raster_nc)
Utils.remove_additional_files(directory=os.path.dirname(input_dem))

# 4. Tracklines to shapefile

In [ ]:
# allowing the user to run all without executing the trackline portion
assert 1==2

In [ ]:
from shapely.geometry import LineString, Point, Polygon
import geopandas as gpd

In [ ]:
# Input path to tracklines folders from Qimera
tracklines_folders_paths = r"C:\Users\ageglio\OneDrive - DOI\Documents - Reef Mapping\Tracks_Data_Release_NLMI_Reefs\Tracklines\*\*"
tracklines_folders = glob.glob(tracklines_folders_paths)

# Define the output folder path for shapefiles
out_folder = "shapefiles"
os.makedirs(out_folder, exist_ok=True)

trackline_folder = tracklines_folders[4] #<------------------------ CHANGE INDEX ID
print("folder path chosen: ", trackline_folder)

## 3b. combine and convert tracklines to a shapefile

In [ ]:
def create_tracklines(trackline_folder):
    tracklines_files = glob.glob(os.path.join(trackline_folder, "*.txt"))
    reef_name = os.path.basename(trackline_folder)
    print("creating tracklines shapefile for: ", reef_name)
    output_shapefile_folder = os.path.join(out_folder, reef_name)
    os.makedirs(output_shapefile_folder, exist_ok=True)
    output_shapefile = os.path.join(output_shapefile_folder, f"{reef_name}.shp")

    # Concatenate all dataframes in the list into a single dataframe
    tracklines = pd.concat([pd.read_csv(file, delimiter=',', header=None) for file in tracklines_files], ignore_index=True)
    tracklines.columns = ["UTC", "X", "Y", "Delta"]

    # Get the lat lon coordinates of the raw combined tracklines file
    xy = GetCoordinates.convert_tracklines_to_lat_lon(tracklines, from_wkt="wgs84_16N_OGC_WKT.txt", wgs84_wkt="wgs84_OGC_WKT.txt")

    # Create a GeoDataFrame and save to a shapefile
    gpd.GeoDataFrame(geometry=[LineString(xy)], crs="EPSG:4326").to_file(output_shapefile)

create_tracklines(trackline_folder)
